In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/asl-alphabet.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Load the data with data augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_ds = train_datagen.flow_from_directory(
    directory='/content/asl_alphabet_train/asl_alphabet_train',
    target_size=(224, 224),
    batch_size=64,
    class_mode='sparse',
    subset='training',
    seed=123
)

val_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

val_ds = val_datagen.flow_from_directory(
    directory='/content/asl_alphabet_train/asl_alphabet_train',
    target_size=(224, 224),
    batch_size=64,
    class_mode='sparse',
    subset='validation',
    seed=123
)

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

base_model.trainable = True

num_classes=29 //29 classes in the dataset

inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=True)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)


In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory='/content/asl_alphabet_test',  # Adjust the directory path as per your test data location
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
    seed=123,
    shuffle=False  # Set shuffle to False for test dataset
)


# Predict labels for test data
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
